# Маркетинговый анализ
***

# Описание проекта

Вы — маркетинговый аналитик развлекательного приложения Procrastinate Pro+. Несмотря на огромные вложения в рекламу, последние несколько месяцев компания терпит убытки. Ваша задача — разобраться в причинах и помочь компании выйти в плюс.
Есть данные о пользователях, привлечённых с 1 мая по 27 октября 2019 года:

- лог сервера с данными об их посещениях,
- выгрузка их покупок за этот период,
- рекламные расходы.

Вам предстоит изучить:

- откуда приходят пользователи и какими устройствами они пользуются,
- сколько стоит привлечение пользователей из различных рекламных каналов;
- сколько денег приносит каждый клиент,
- когда расходы на привлечение клиента окупаются,
- какие факторы мешают привлечению клиентов.

## Оглавление<a class="anchor" id="TOC"></a>
***

* **[I. Загрузка и подготовка данных](#1)**
    - [1) Импортируем библиотеки, объявим класс MetricCalculator для анализа](#1_1)
    - [2) Прочитаем данные](#1_2)
    - [3) Выведем первые и последние пять строк каждой из таблиц, общую информацию](#1_3)
    - [Вывод](#1_conclusion)
* **[II. Исследовательский анализ данных](#2)**
    - [Вывод](#2_conclusion)
* **[III. Маркетинговый анализ](#3)**
    - [1) Выделим ключевые значения для расчетов](#3_1)
    - [2) Подсчитаем минимальный средний объём сырья в месторождениях региона, достаточный для его разработки](#3_2)
    - [3) Подготовим функцию для расчёта прибыли по набору отобранных месторождений и предсказаний модели](#3_3)
    - [Вывод](#3_conclusion)
* **[IV. Расчёт прибыли и рисков](#4)**
    - [1) Найдем распределение прибыли с помощью Bootstrap](#4_1)
    - [2) Подсчитаем среднюю прибыль, 95-% доверительный интервал и риск убытков](#4_2)
    - [Вывод](#4_conclusion)

# I. Загрузка и подготовка данных<a class='anchor' id='1'>
***

## 1) Импортируем библиотеки, объявим класс MetricCalculator для анализа<a class="anchor" id="1_1"></a>

In [ ]:
# тут импорты
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
from calculator.calculator import MetricCalculator
from IPython.display import display

In [ ]:
# тут класс

In [ ]:
# тут полезное
pd.set_option('display.float_format', lambda x: '%.2f' % x)

[В оглавление](#TOC)

## 2) Прочитаем данные<a class="anchor" id="1_2"></a>

### - описание данных

Файл visits_info_short.csv хранит лог сервера с информацией о посещениях сайта, orders_info_short.csv — информацию о покупках, а costs_info_short.csv — информацию о расходах на рекламу.


Структура visits_info_short.csv|Описание|Структура orders_info_short.csv|Описание|Структура costs_info_short.csv|Описание
:-----|:-----|:-----|:-----|:-----|:-----  
User Id|уникальный идентификатор пользователя|User Id|уникальный идентификатор пользователя|Channel|идентификатор рекламного источника
Region|страна пользователя|Event Dt|дата и время покупки|Dt|дата проведения рекламной кампании
Device|тип устройства пользователя|Revenue|сумма заказа|Costs|расходы на эту кампанию
Channel|идентификатор источника перехода| | | | 
Session Start|дата и время начала сессии| | | | 
Session End|дата и время окончания сессии| | | | 

### - создадим экземпляр класса с путями до данных, приведем наименование колонок к едином формату, установим нужный тип данных

In [ ]:
calc = MetricCalculator('datasets/visits_info_short.csv', 'datasets/orders_info_short.csv', 'datasets/costs_info_short.csv')

In [ ]:
calc.columns_fixer()

Cоздан экземпляр класса:
- данные прочитаны, 
- наименование колонок приведено к единому формату,
- колонки с датами приведены к соответствующему формату.

Датасеты доступны через экземпляр класса по именам **visits, orders, costs**.

## 3) Выведем первые пять первых и последних строк каждой из таблиц, общую информацию<a class="anchor" id="1_3"></a>

### - visits - посещение сайта

In [ ]:
calc.visits

In [ ]:
calc.visits.info()

&#9889; **Вывод:** в датасете с информацией о посещениях сайта 309 901 строка, без пропусков, без проблем с типами данных.

### - orders - информация о покупках

In [ ]:
calc.orders

In [ ]:
calc.orders.info()

&#9889; **Вывод:** в датасете с информацией о покупках 40 212 строк (13% от числа визитов), без пропусков, без проблем с типами данных.

### - costs - информация о расходах

In [ ]:
calc.costs

In [ ]:
calc.costs.info()

&#9889; **Вывод:** в датасете с информацией расходах 1800, без пропусков, без проблем с типами данных.

### - проверим наличие полных дубликатов

In [ ]:
datasets = [calc.visits, calc.orders, calc.costs]
duplicated_sum = 0
for dataset in datasets:
    duplicated_sum += (dataset.duplicated() == True).sum()
print(duplicated_sum)

Полных дубликатов нет

## Вывод<a class="anchor" id="1_conclusion"></a>

1. Прочитали данные с помощью специального класса, наименование колонок приведено к единому формату, колонки с датами приведены к соответствующему формату. Датасеты доступны через экземпляр класса по именам visits, orders, costs.
2. Во всех датасетах нет проблем с типами данных, полные дубликаты отсутствуют, нет пропусков.

[В оглавление](#TOC)

# II. Исследовательский анализ данных<a class='anchor' id='2'>
***

**В отношении user_id считать статистику неинформативно - поэтому исключим данный столбец**

### - visits - посещение сайта

In [ ]:
calc.visits.describe(exclude=['int64']).T.fillna('---')

Проверим записи о пользователях из каких стран содержит таблица:

In [ ]:
unique_users_by_сountries = sorted(calc.visits.groupby('region')['user_id'].nunique())
countries_count = calc.visits.groupby('region').agg({'user_id': 'count'}).reset_index().sort_values(by='user_id')
countries_count['unique_id'] = unique_users_by_сountries
countries_count['% of observations'] = (countries_count['user_id'] / len(calc.visits)) * 100
countries_count['% of unique id'] = (countries_count['unique_id'] / sum(unique_users_by_сountries)) * 100
countries_count

Всего уникальных пользователей:

In [ ]:
sum(unique_users_by_сountries)

Из 309 901 наблюдений - 207 327 (67%) сделаны в отношении пользователей из США. Из 309 901 наблюдений - 150 008 об уникальных пользователях, процентное соотношение пользователей из различных регионов сохраняется и с учетом только уникальных пользователей.

Проверим сведения об устройствах: 

In [ ]:
unique_devices = sorted(calc.visits.groupby('device')['user_id'].nunique())
device_count = calc.visits.groupby('device').agg({'user_id': 'count'}).reset_index().sort_values(by='user_id')
device_count['unique_id'] = unique_devices
device_count['% of observations'] = (device_count['user_id'] / len(calc.visits)) * 100
device_count['% of unique id'] = (device_count['unique_id'] / sum(unique_devices)) * 100
device_count

Наблюдения об устройствах распределены равномернее, чем между странами - больше всего пользователей iPhone (36%), пользователей Mac (20%), PC (20%) и Andoid (23%).

Визуализируем соотношение устройств по числу пользователей:

In [ ]:
calc.square_plot(calc.visits, 'device', title='Соотношение различных устройств клиентов')

In [ ]:
title = 'Устройства по числу пользователей'
xlabel = 'Устройство'
ylabel = 'Число пользователей'
data = device_count.sort_values(by='user_id', ascending=False)
calc.sns_catplot(x='device', y='user_id', data=data, title=title, xlabel=xlabel, ylabel=ylabel)

Всего уникальных устройств:

In [ ]:
sum(unique_devices)

С учетом того, что количество уникальных устройств превышает число уникальных пользователей - некоторые пользователи пользуются услугой с различных устройств.

In [ ]:
unique_channels = sorted(calc.visits.groupby('channel')['user_id'].nunique())
channel_count = calc.visits.groupby('channel').agg({'user_id': 'count'}).reset_index().sort_values(by='user_id')
channel_count['unique_id'] = unique_channels
channel_count['% of observations'] = (channel_count['user_id'] / len(calc.visits)) * 100
channel_count['% of unique id'] = (channel_count['unique_id'] / sum(unique_channels)) * 100
channel_count

In [ ]:
calc.square_plot(calc.visits, 'channel', title='Соотношение различных источников привлечения клиентов')

In [ ]:
title = 'Пользователи по каждому из каналов привлечения'
xlabel = 'Рекламный канал'
ylabel = 'Число привлеченных пользователей'
data = channel_count.sort_values(by='user_id', ascending=False)
calc.sns_catplot(x='channel', y='user_id', data=data, title=title, xlabel=xlabel, ylabel=ylabel)

&#9889; **Вывод**

В датасете о посещениях сайта:

- сведения о 4 регионах, больше всего наблюдений о пользователях из США - 207 327 из 309 901 наблюдения (67%), больше всего пользовавшихся iPhone - 112 603 из 309 901 наблюдения (36%),
- более трети пользователей пришли напрямую - 107 760 из 309 901 (36%),
- помимо прямой покупки, с большим отрывом лидируют источники привлечения - TipTop (54 794, 18%) и FaceBoom (20204, 16%),
- самое раннее наблюдение о первом действии - 01.05.2019 года, самое позднее - 31.10.2019 года.

### - orders - информация о покупках

In [ ]:
calc.orders.describe(include=['float64']).T

In [ ]:
calc.orders.describe(include=['object', 'datetime64[ns]']).T

С учетом того, что 75%-процентиль - 4.99 (совпадающий с медианой, 25%-процентилем и минимуом), максимальная сумма выручки - 49.99, проверим какие значения выручки есть в датасете:

In [ ]:
sorted(calc.orders.revenue.unique())

Всего пять значений стоимости услуги. Проверим сколько пользователей платят больше 4.99, возможно, таких пользователей стоит рассматривать отдельно:

In [ ]:
max_revenue = calc.orders.groupby('revenue').agg({'user_id': 'count'}).reset_index()
max_revenue['%'] = (max_revenue['user_id'] / len(calc.orders)) * 100
max_revenue

Примерно 5% пользователей платят больше базовой стоимости. Посчитаем общее число таких пользователей:

In [ ]:
sum(max_revenue[max_revenue['revenue'] > 4.99]['user_id'])

&#9889; **Вывод**

В датасете с информацией о покупках:

- остутствуют сведения о валюте в которой выражена выручка,
- принимая во внимание, что 67% пользователей из США, можно предположить, что валюта выручки - доллары США,
- выявлено 5 различных стоимостей услуг - 4.99, 5.99, 9.99, 19.99, 49.99.
- минимальная сумма выручки - 4.99, средняя - 5.37 (при медианной и 75%-процентиле - 4.99), максимальная - 49.99,
- из 40 185 платящих пользователей - 1581 (4%) платит больше базового тарифа,
- самое раннее наблюдение о первом действии совпадает с датасетом о посещениях - 01.05.2019 года, самое позднее - 31.10.2019 года.

### - costs - информация о расходах

In [ ]:
calc.costs.describe().T

Минимально - 0.80, средняя стоимость источника - 58.61 (при медианой - 12.29 и 75%-процентиле равным 33.60). Есть очень дорогие источники. Проверим среднюю стоимость привлечения по каждому источнику: 

In [ ]:
channels_sum_costs = calc.costs.groupby('channel')['costs'].sum()
channels_mean_costs = calc.costs.groupby('channel').agg({'costs': 'mean'}).reset_index()
channels_mean_costs['cost_sum_by_channel'] = channels_sum_costs.values
channels_mean_costs['% of total costs'] = (channels_mean_costs['cost_sum_by_channel'] / sum(calc.costs['costs'])) * 100
channels_mean_costs.sort_values(by='costs')

In [ ]:
data = channels_mean_costs.sort_values(by='costs', ascending=False)
ylabel = 'Средняя стоимость'
xlabel = 'Источник привлечения'
title = 'Средняя стоимость привлечения по источнику'
calc.sns_catplot(x='channel', y='costs', data=data, xlabel=xlabel, ylabel=ylabel, title=title)

TipTop (304.17) и FaceBoom (180.25) лидируют с отрывом, самый дешевый источник - YRabbit (5.25). Проверим к кому обращались за рекламой чаще:

In [ ]:
calc.costs.describe(include=['object', 'datetime64[ns]']).T

In [ ]:
calc.costs.groupby('channel').agg({'channel': 'count'})

К каждому из источников привлечения по 180 раз.

&#9889; **Вывод:**

В датасете с информацией о покупках:

- всего десять каналов привлечения (помимо естественного, без посредников),
- средняя стоимость привлечения клиента - 58.61 (при медианой - 12.29 и 75%-процентиле равным 33.60),
- TipTop (304.17) и FaceBoom (180.25) лидируют с отрывом, самый дешевый - YRabbit (5.25),
- к каждому из источников обращались по 180 раз,
- самое раннее наблюдение о понесенных расходах - 01.05.2019 года, самое позднее - 27.10.2019 года.

### - объеденим полученные таблицы с расходами на источник и привлеченными по источнику

In [ ]:
channels_grouped = channels_mean_costs.merge(channel_count.query('channel != "organic"'), on='channel').drop(['unique_id', '% of unique id'], axis=1)
channels_grouped = channels_grouped.rename(columns={'user_id': 'total_customers'}).sort_values(by='costs')
channels_grouped

In [ ]:
channels_grouped.query('channel in ["FaceBoom", "TipTop"]').cost_sum_by_channel.sum()

In [ ]:
channels_grouped.query('channel in ["FaceBoom", "TipTop"]')['% of total costs'].sum()

In [ ]:
channels_grouped.query('channel not in ["FaceBoom", "TipTop"]').cost_sum_by_channel.sum()

In [ ]:
channels_grouped.query('channel not in ["FaceBoom", "TipTop"]').total_customers.sum() 

In [ ]:
channels_grouped.query('channel not in ["FaceBoom", "TipTop"]').total_customers.sum() / 309901

In [ ]:
channels_grouped.query('channel not in ["FaceBoom", "TipTop"]')['% of total costs'].sum()

## Вывод<a class="anchor" id="2_conclusion"></a>

Исходя из представленных данных и описаний данных, можно сделать выводы:

- представлены сведения о 4 регионах, больше всего наблюдений о пользователях из США - 207 327 из 309 901 наблюдения (67%), больше всего пользовавшихся iPhone - 112 603 из 309 901 наблюдения (36%),
- минимальная сумма выручки - 4.99, средняя - 5.37, максимальная - 49.99,
- из 40 185 платящих пользователей - 1581 (4%) платит больше базового тарифа,
- более трети пользователей пришли напрямую - 107 760 из 309 901 (36%),
- помимо прямой покупки, с большим отрывом лидируют источники привлечения - TipTop (54 794, 18%) и FaceBoom (20204, 16%), они же и являются самыми дорогими со средней стоимостью 304.17 и 180.25, соответственно,
- 83% (87 196) рекламного бюджета потрачено на TipTop и FaceBoom, которые вместе привели 103 816 пользователей (33 %). Вместе с тем, оставшиеся 17% (18 300) бюджета направлено на 8 источников принесших 31% (98 325). Следовательно, при соотношении трат 5 к 1 - результаты сопоставимые, что свидетельствует о неэффективности TipTop и FaceBoom.



[В оглавление](#TOC)

# III. Маркетинговый анализ<a class='anchor' id='3'>
***

## 1) Получим профили пользователей для анализа<a class="anchor" id="1_1"></a>

In [ ]:
profiles = calc.get_profiles()

In [ ]:
profiles

In [ ]:
from datetime import datetime
from matplotlib import pyplot as plt

In [ ]:
observation_date = datetime(2019, 11, 1).date()
horizon_days = 14

In [ ]:
profiles.pivot_table(
    index='dt', columns='channel', values='acquisition_cost', aggfunc='mean'
).plot(grid=True, figsize=(10, 5))
plt.ylabel('CAC, $')
plt.xlabel('Дата привлечения')
plt.title('Динамика САС по каналам привлечения')
plt.show()

In [ ]:
ltv_raw, ltv_grouped, ltv_hist, roi_grouped, roi_hist = calc.get_ltv(profiles, observation_date, horizon_days, dimensions=['channel'])

In [ ]:
calc.plot_ltv_roi(ltv_grouped, ltv_hist, roi_grouped, roi_hist, horizon_days, window=14) 

In [ ]:
retention_raw, retention_grouped, retention_hist = calc.get_retention(profiles, observation_date, horizon_days)

In [ ]:
calc.plot_retention(retention_grouped, retention_hist, horizon_days)

In [ ]:
cr_raw, cr_grouped, cr_hist = calc.get_conversion(profiles, observation_date, horizon_days)

In [ ]:
calc.plot_conversion(cr_grouped, cr_hist, horizon_days)

In [ ]:
ltv_raw, ltv_grouped, ltv_hist, roi_grouped, roi_hist = calc.get_ltv(profiles, observation_date, horizon_days, dimensions=['channel'])

In [ ]:
cac_hist = roi_hist[['cac']]

In [ ]:
columns = [name for name in cac_hist.index.names if name not in ['dt']]
columns

In [ ]:
filtered_data = cac_hist.pivot_table(index='dt', columns=columns, values='cac', aggfunc='mean').plot(grid=True)

In [ ]:
cac_hist

In [ ]:
calc.plot_ltv_roi(ltv_grouped, ltv_hist, roi_grouped, roi_hist, horizon_days) 

In [ ]:
ltv_raw, ltv_grouped, ltv_hist, roi_grouped, roi_hist = calc.get_ltv(profiles.query('channel == "FaceBoom"'), observation_date, horizon_days, dimensions=['region'])

In [ ]:
calc.plot_ltv_roi(ltv_grouped, ltv_hist, roi_grouped, roi_hist, horizon_days, window=14) 

In [ ]:
ltv_raw, ltv_grouped, ltv_hist, roi_grouped, roi_hist = calc.get_ltv(profiles.query('channel == "FaceBoom"'), observation_date, horizon_days)

In [ ]:
calc.plot_ltv_roi(ltv_grouped, ltv_hist, roi_grouped, roi_hist, horizon_days) 

In [ ]:
ltv_raw, ltv_grouped, ltv_hist, roi_grouped, roi_hist = calc.get_ltv(profiles, observation_date, horizon_days, dimensions=['payer'])

In [ ]:
calc.plot_ltv_roi(ltv_grouped, ltv_hist, roi_grouped, roi_hist, horizon_days) 